In [2]:
import pandas as pd
from datetime import datetime
from os.path import join
import json

In [3]:
DATA_DIR = '../public'

In [4]:
df_raw = pd.read_json(join(DATA_DIR, 'mtbs.json'))

## Save Fire Year and Burned Acreage

In [74]:
df = df_raw.copy()

In [75]:
df = df.sort_values('burned_acreage', ascending=False)

In [76]:
df = df[['ignition_year', 'burned_acreage', 'incident_name']]

In [77]:
fires_by_decade = {
    1990: df[(df.ignition_year >= 1990) & (df.ignition_year < 2000)],
    2000: df[(df.ignition_year >= 2000) & (df.ignition_year < 2010)],
    2010: df[(df.ignition_year >= 2000) & (df.ignition_year < 2020)],
}

In [78]:
fires = {
    'name': 'fires',
    'children': []
}

In [79]:
for decade, v in fires_by_decade.items():
    d = {
        'name': decade,
        'children': []
    }
    v = v.nlargest(10, 'burned_acreage')
    d['children'].extend(
        v.apply(lambda row: {
            'name': row.incident_name.title(),
            'acresBurned': row.burned_acreage
        }, axis='columns')
    )
    fires['children'].append(d)

In [80]:
with open(join(DATA_DIR, 'pack_burn_by_decade.json'), 'w') as fp:
    json.dump(fires, fp)

## Save Fire Year and Burned Acreage, By Month

### Pack Decades and acres burned by month

In [15]:
df = df_raw.copy()

In [17]:
df['month'] = df.ignition_date.apply(lambda d: datetime.strptime(d, '%Y-%m-%d').strftime("%B"))

In [20]:
fires_by_decade = {
    1990: df[(df.ignition_year >= 1990) & (df.ignition_year < 2000)],
    2000: df[(df.ignition_year >= 2000) & (df.ignition_year < 2010)],
    2010: df[(df.ignition_year >= 2000) & (df.ignition_year < 2020)],
}

In [ ]:
fires = {
    'name': 'fires',
    'children': []
}

In [33]:
x = fires_by_decade[2010].groupby('month')['burned_acreage'].sum()

In [41]:
x.to_frame().reset_index

<bound method DataFrame.reset_index of            burned_acreage
month                    
April            13484722
August           22946090
December           746670
February          2452721
January           1212869
July             36110976
June             31860548
March             7652398
May               9363492
November          1309632
October           2639657
September         4805098>

In [ ]:
for decade, v in fires_by_decade.items():
    d = {
        'name': decade,
        'children': []
    }
    v = v.groupby('month')['burned_acreage'].sum()
    v = v.to_frame().reset_index()
    d['children'].extend(
        v.apply(lambda row: {
            'name': row.month,
            'acresBurned': row.burned_acreage
        }, axis='columns')
    )
    fires['children'].append(d)

In [ ]:
with open(join(DATA_DIR, 'pack_burn_by_decade.json'), 'w') as fp:
    json.dump(fires, fp)

In [371]:
df_by_month = [
    df[df.month == month].drop(columns=['month'])
    for month in range(1, 13)
]

In [372]:
for month, df in enumerate(df_by_month):
    df['intra_year_index'] = df.groupby('ignition_year').cumcount()
    df_by_month[month] = df

In [373]:
for idx, df in enumerate(df_by_month):
    df = df.rename(columns={
        'burned_acreage': 'acresBurned',
        'intra_year_index': 'intraYearIndex',
        'ignition_year': 'year'
    })
    df_by_month[idx] = df

In [374]:
with open(join(DATA_DIR, 'timelapse_by_month.json'), 'w') as outfile:
    outfile.write(json.dumps([df.to_dict(orient='records') for df in df_by_month]))